In [ ]:
!nvidia-smi

In [ ]:
# ====== Cell 1: Install dependencies ======
pip -q install -U openmim
mim -q install mmengine "mmcv==2.1.0"
pip -q install mmdet

# Clone MMDetection repo for using the tools scripts (test.py)
git clone -q https://github.com/open-mmlab/mmdetection.git
pip -q install -e mmdetection

python - << 'PY'
import mmcv, mmengine, mmdet, torch
print("✓ Torch:", torch.__version__)
print("✓ MMCV :", mmcv.__version__)
print("✓ MMEngine:", mmengine.__version__)
print("✓ MMDetection:", mmdet.__version__)
PY

In [ ]:
# ====== Cell 2: Download MS R-CNN config and checkpoint ======
mim download mmdet --config ms-rcnn_r50_fpn_1x_coco --dest .

python - << 'PY'
import os, glob, json, pprint
from pathlib import Path

# Edit these if your COCO path is different
COCO_ROOT = Path("/kaggle/input/coco-2017-dataset/coco2017")
ANN_FILE  = COCO_ROOT / "annotations/instances_val2017.json"
IMG_DIR   = COCO_ROOT / "val2017"

assert ANN_FILE.exists(), f"COCO annotations not found: {ANN_FILE}"
assert IMG_DIR.exists(),  f"COCO images dir not found: {IMG_DIR}"

# Resolve downloaded config/ckpt
cfg_path = Path(".") / "ms-rcnn_r50_fpn_1x_coco.py"
ckpt = sorted(Path(".").glob("ms-rcnn_r50_fpn_1x_coco_*.pth"))
assert cfg_path.exists(), "Config not found (download failed?)"
assert len(ckpt) > 0, "Checkpoint not found (download failed?)"
ckpt_path = ckpt[0]

print("✓ Config:", cfg_path)
print("✓ Checkpoint:", ckpt_path)
print("✓ COCO ann:", ANN_FILE)
print("✓ COCO img:", IMG_DIR)

# Persist for later cells
meta = {
    "cfg_path": str(cfg_path.resolve()),
    "ckpt_path": str(ckpt_path.resolve()),
    "ann_file": str(ANN_FILE.resolve()),
    "img_dir": str(IMG_DIR.resolve()),
    "outfile_prefix": str(Path("./results/msrcnn_r50_fpn_1x").resolve())
}
os.makedirs("results", exist_ok=True)
with open("results/run_meta.json", "w") as f:
    json.dump(meta, f, indent=2)
pprint.pp(meta)
PY

In [ ]:
# ====== Cell 3: Run evaluation and export COCO JSONs ======
python mmdetection/tools/test.py \
  "$(jq -r .cfg_path results/run_meta.json)" \
  "$(jq -r .ckpt_path results/run_meta.json)" \
  --eval bbox segm \
  --cfg-options \
    test_dataloader.dataset.ann_file="$(jq -r .ann_file results/run_meta.json)" \
    test_dataloader.dataset.data_prefix.img="$(jq -r .img_dir results/run_meta.json)" \
    test_evaluator.outfile_prefix="$(jq -r .outfile_prefix results/run_meta.json)"

In [ ]:
# ====== Cell 4: Parse MMDet log to record baseline metrics ======
import re, json, os, pathlib

logfile = "results/baseline_log.txt"
# Kaggle prints the previous cell output to stdout, not an easy file.
# We will re-evaluate using pycocotools below to capture the exact numbers,
# and save both bbox & segm baselines.

from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

with open("results/run_meta.json") as f:
    meta = json.load(f)

ann = meta["ann_file"]
bbox_json = meta["outfile_prefix"] + ".bbox.json"
segm_json = meta["outfile_prefix"] + ".segm.json"

coco_gt = COCO(ann)

def eval_json(json_path, iouType):
    coco_dt = coco_gt.loadRes(json_path)
    ev = COCOeval(coco_gt, coco_dt, iouType)
    ev.evaluate(); ev.accumulate(); ev.summarize()
    return {
        "AP@[.50:.95]": ev.stats[0],
        "AP@.50": ev.stats[1],
        "AP@.75": ev.stats[2],
        "AP_small": ev.stats[3],
        "AP_medium": ev.stats[4],
        "AP_large": ev.stats[5],
    }

print("== Baseline (MS R-CNN) — bbox ==")
bbox_metrics = eval_json(bbox_json, "bbox")
print("\n== Baseline (MS R-CNN) — segm ==")
segm_metrics = eval_json(segm_json, "segm")

baseline = {
    "model": "MS R-CNN R50-FPN (1x)",
    "framework": "MMDetection",
    "dataset": "COCO 2017 val",
    "bbox": bbox_metrics,
    "segm": segm_metrics,
    "files": {"bbox_json": bbox_json, "segm_json": segm_json}
}
with open("results/baseline_metrics.json", "w") as f:
    json.dump(baseline, f, indent=2)

print("\nSaved → results/baseline_metrics.json")

In [ ]:
# ====== Cell 5: Rescore using a simple shape prior and re-evaluate (segm) ======
import json, copy, numpy as np, os
from tqdm import tqdm
from pycocotools import mask as mask_utils
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

with open("results/run_meta.json") as f:
    meta = json.load(f)

ann = meta["ann_file"]
segm_json = meta["outfile_prefix"] + ".segm.json"
assert os.path.exists(segm_json), "Missing baseline segm.json"

with open(segm_json) as f:
    segm_dets = json.load(f)

def bbox_area(b):
    # [x,y,w,h]
    return max(0.0, b[2]) * max(0.0, b[3])

def compute_ratio_and_rescore(dets, alpha):
    out = []
    for d in dets:
        rle = d.get("segmentation", None)
        if not rle:
            continue
        # pycocotools may return counts as str already; ensure proper format
        if isinstance(rle.get("counts", None), str):
            rle_use = rle
        else:
            # ensure Fortran order if decoding was needed; here we trust JSON as-is
            rle_use = rle
        try:
            m_area = float(mask_utils.area(rle_use))
        except Exception:
            # If decoding fails due to malformed counts, skip
            m_area = 0.0
        b_area = float(bbox_area(d["bbox"])) + 1e-6
        ratio = max(1e-6, min(1.0, m_area / b_area))  # clamp to (0,1]
        s = float(d["score"])
        s_new = (s ** alpha) * (ratio ** (1.0 - alpha))
        d2 = copy.deepcopy(d)
        d2["score"] = float(s_new)
        out.append(d2)
    return out

def eval_segm(json_path):
    coco_gt = COCO(ann)
    coco_dt = coco_gt.loadRes(json_path)
    ev = COCOeval(coco_gt, coco_dt, "segm")
    ev.evaluate(); ev.accumulate(); ev.summarize()
    return {
        "AP@[.50:.95]": ev.stats[0],
        "AP@.50": ev.stats[1],
        "AP@.75": ev.stats[2],
        "AP_small": ev.stats[3],
        "AP_medium": ev.stats[4],
        "AP_large": ev.stats[5],
    }

alphas = [0.25, 0.5, 0.75]
results = {}
for a in alphas:
    rescored = compute_ratio_and_rescore(segm_dets, alpha=a)
    out_path = f"results/msrcnn_r50_fpn_1x.segm.alpha{a:.2f}.json"
    with open(out_path, "w") as f:
        json.dump(rescored, f)
    print(f"\n== Rescored segm eval (alpha={a:.2f}) ==")
    results[f"alpha={a:.2f}"] = eval_segm(out_path)

with open("results/shape_prior_ablation.json", "w") as f:
    json.dump(results, f, indent=2)

print("\nSaved → results/shape_prior_ablation.json")

In [ ]:
# ====== Cell 6: Final summary table ======
import json, pandas as pd, numpy as np

with open("results/baseline_metrics.json") as f:
    base = json.load(f)
with open("results/shape_prior_ablation.json") as f:
    abla = json.load(f)

rows = []
rows.append(("Baseline (MS R-CNN)", 
             base["segm"]["AP@[.50:.95]"], 
             base["segm"]["AP@.50"], 
             base["segm"]["AP@.75"], 
             base["segm"]["AP_small"], 
             base["segm"]["AP_medium"], 
             base["segm"]["AP_large"]))

for k, m in abla.items():
    rows.append((f"Rescore {k}", m["AP@[.50:.95]"], m["AP@.50"], m["AP@.75"], m["AP_small"], m["AP_medium"], m["AP_large"]))

df = pd.DataFrame(rows, columns=["Method","AP@[.50:.95]","AP@.50","AP@.75","AP_small","AP_medium","AP_large"])
pd.options.display.float_format = "{:,.4f}".format
df.sort_values(by="AP@[.50:.95]", ascending=False, inplace=True)
df.reset_index(drop=True, inplace=True)
df

In [ ]:
# ====== Visual A: use MMDetection high-level API ======
import os, cv2, random, glob, json
import matplotlib.pyplot as plt
from mmengine.config import Config
from mmdet.apis import init_detector, inference_detector

with open("results/run_meta.json") as f:
    meta = json.load(f)

cfg_path   = meta["cfg_path"]
ckpt_path  = meta["ckpt_path"]
img_dir    = meta["img_dir"]

cfg = Config.fromfile(cfg_path)
# Make sure the dataset root in config doesn't break init; we only use the model weights here.
model = init_detector(cfg, ckpt_path, device='cuda:0')

def show_image(img_path, score_thr=0.5):
    result = inference_detector(model, img_path)
    vis = model.show_result(
        img_path, result, score_thr=score_thr, show=False, wait_time=0
    )
    vis_rgb = cv2.cvtColor(vis, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(10,10))
    plt.imshow(vis_rgb); plt.axis('off'); plt.title(os.path.basename(img_path))

images = glob.glob(os.path.join(img_dir, "*.jpg"))
for p in random.sample(images, 3):
    show_image(p, score_thr=0.5)

In [ ]:
# ====== Visual B: draw from COCO JSON (segm.json) ======
import os, json, random, cv2, numpy as np, matplotlib.pyplot as plt
from collections import defaultdict
from pycocotools.coco import COCO
from pycocotools import mask as mask_utils

with open("results/run_meta.json") as f:
    meta = json.load(f)

ann_file   = meta["ann_file"]
img_dir    = meta["img_dir"]
segm_json  = meta["outfile_prefix"] + ".segm.json"  # baseline segm
assert os.path.exists(segm_json)

coco = COCO(ann_file)
with open(segm_json) as f:
    dets = json.load(f)

# Build catId -> name for pretty labels
catId_to_name = {c['id']: c['name'] for c in coco.loadCats(coco.getCatIds())}

# index detections by image_id for quick access
by_img = defaultdict(list)
for d in dets:
    by_img[int(d['image_id'])].append(d)

def draw_instance(ax, img_h, img_w, det, color):
    # bbox
    x,y,w,h = det['bbox']
    rect = plt.Rectangle((x,y), w, h, fill=False, edgecolor=color, linewidth=2)
    ax.add_patch(rect)

    # mask
    rle = det.get('segmentation')
    if isinstance(rle, dict):
        m = mask_utils.decode(rle)  # (H, W, 1) or (H,W)
        if m.ndim == 3: m = m[:,:,0]
        # overlay as transparent color
        masked = np.zeros((img_h,img_w,4), dtype=np.uint8)
        masked[m>0] = (*[int(255*c) for c in color_to_rgb(color)], 70)
        ax.imshow(masked)

    # label
    cat = catId_to_name.get(det['category_id'], str(det['category_id']))
    ax.text(x, y-2, f"{cat} {det['score']:.2f}", color=color, fontsize=10,
            bbox=dict(facecolor='black', alpha=0.3, pad=1, edgecolor='none'))

def color_to_rgb(matplotlib_color):
    # convert matplotlib color to rgb (0..1)
    from matplotlib.colors import to_rgb
    return to_rgb(matplotlib_color)

def visualize_image(image_id, score_thr=0.5, top_k=30, cmap=None):
    img_info = coco.loadImgs([image_id])[0]
    img_path = os.path.join(img_dir, img_info['file_name'])
    im = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
    H,W = im.shape[:2]

    fig, ax = plt.subplots(1,1, figsize=(10,10))
    ax.imshow(im); ax.axis('off'); ax.set_title(f"image_id={image_id}")

    # pick a palette of distinct colors
    colors = plt.cm.get_cmap('tab20', 20)
    dets_img = sorted([d for d in by_img[image_id] if d['score']>=score_thr], key=lambda d: -d['score'])[:top_k]
    for i, d in enumerate(dets_img):
        draw_instance(ax, H, W, d, colors(i % 20))

    plt.show()

# show 3 random images with detections
img_ids = coco.getImgIds()
for iid in random.sample(img_ids, 3):
    visualize_image(iid, score_thr=0.5)

In [ ]:
# ====== Visual C: side-by-side baseline vs rescored ======
import os, json, random, cv2, numpy as np, matplotlib.pyplot as plt
from collections import defaultdict
from pycocotools.coco import COCO
from pycocotools import mask as mask_utils

with open("results/run_meta.json") as f:
    meta = json.load(f)

ann_file   = meta["ann_file"]
img_dir    = meta["img_dir"]
base_json  = meta["outfile_prefix"] + ".segm.json"
rescore_json = "results/msrcnn_r50_fpn_1x.segm.alpha0.50.json"  # choose the α you ran

coco = COCO(ann_file)
with open(base_json) as f: base_dets = json.load(f)
with open(rescore_json) as f: resc_dets = json.load(f)

def index_by_img(dets):
    b = defaultdict(list)
    for d in dets:
        b[int(d['image_id'])].append(d)
    return b

base_by_img = index_by_img(base_dets)
resc_by_img = index_by_img(resc_dets)

def draw(ax, img, dets, score_thr=0.5, top_k=25):
    H,W = img.shape[:2]
    ax.imshow(img); ax.axis('off')
    colors = plt.cm.get_cmap('tab20', 20)
    for i, d in enumerate(sorted([d for d in dets if d['score']>=score_thr], key=lambda x:-x['score'])[:top_k]):
        x,y,w,h = d['bbox']; col = colors(i%20)
        rect = plt.Rectangle((x,y), w,h, fill=False, edgecolor=col, linewidth=2)
        ax.add_patch(rect)
        rle = d.get('segmentation')
        if isinstance(rle, dict):
            m = mask_utils.decode(rle)
            if m.ndim==3: m = m[:,:,0]
            overlay = np.zeros((H,W,4), dtype=np.uint8)
            rgb = [int(255*v) for v in plt.cm.get_cmap('tab20c')(i%20)[:3]]
            overlay[m>0] = (*rgb, 70)
            ax.imshow(overlay)
        ax.text(x, y-2, f"{d['category_id']} {d['score']:.2f}", color='w',
                bbox=dict(facecolor='k', alpha=0.4, pad=1, edgecolor='none'), fontsize=9)

# Pick a random image with non-empty detections
img_ids = [iid for iid in coco.getImgIds() if iid in base_by_img and iid in resc_by_img]
iid = random.choice(img_ids)
img_info = coco.loadImgs([iid])[0]
img = cv2.cvtColor(cv2.imread(os.path.join(img_dir, img_info['file_name'])), cv2.COLOR_BGR2RGB)

fig, axes = plt.subplots(1,2, figsize=(18,9))
axes[0].set_title("Baseline MS R-CNN (segm)")
draw(axes[0], img, base_by_img[iid], score_thr=0.5)

axes[1].set_title("Rescored (alpha=0.50)")
draw(axes[1], img, resc_by_img[iid], score_thr=0.5)
plt.show()

In [ ]:
# ====== Visual D1: scatter of score vs shape ratio ======
import json, numpy as np, matplotlib.pyplot as plt
from pycocotools import mask as mask_utils

with open("results/run_meta.json") as f:
    meta = json.load(f)
segm_json = meta["outfile_prefix"] + ".segm.json"

with open(segm_json) as f:
    dets = json.load(f)

def bbox_area(b): return max(0.0,b[2])*max(0.0,b[3])
xs, ys = [], []
for d in dets[:20000]:  # subsample for speed
    rle = d.get("segmentation")
    if not isinstance(rle, dict): continue
    try:
        m_area = float(mask_utils.area(rle))
    except Exception:
        continue
    b_area = bbox_area(d['bbox']) + 1e-6
    ratio = max(1e-6, min(1.0, m_area / b_area))
    xs.append(d['score'])
    ys.append(ratio)

plt.figure(figsize=(6,5))
plt.scatter(xs, ys, s=2, alpha=0.2)
plt.xlabel("Original score"); plt.ylabel("Mask area / BBox area (shape ratio)")
plt.title("Score vs Shape Ratio (subsample)")
plt.grid(True, alpha=0.3); plt.show()

In [ ]:
# ====== Visual D2: PR curve for one category, baseline vs rescored ======
import json, numpy as np, matplotlib.pyplot as plt
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

with open("results/run_meta.json") as f:
    meta = json.load(f)

ann = meta["ann_file"]
base_json = meta["outfile_prefix"] + ".segm.json"
resc_json = "results/msrcnn_r50_fpn_1x.segm.alpha0.50.json"

coco_gt = COCO(ann)
catIds = coco_gt.getCatIds()
# choose a frequent class (e.g., person=1 if using COCO ids)
target_cat = 1 if 1 in catIds else catIds[0]

def pr_curve(json_file, cat_id):
    coco_dt = coco_gt.loadRes(json_file)
    ev = COCOeval(coco_gt, coco_dt, "segm")
    ev.params.catIds = [cat_id]
    ev.evaluate(); ev.accumulate()
    # precision: [TxRxKxAxM] (IoU thresholds x recall x categories x area x maxDets)
    # take AP at IoU=0.50:0.95 averaged (index : over all iou thresholds)
    precision = ev.eval['precision']  # shape [T,R,K,A,M]
    # Average over IoU thresholds and areas, take first maxDet.
    p = precision.mean(axis=(0,3))[:,0]  # [R]
    recalls = ev.params.recThrs  # [R]
    return recalls, p

r_base, p_base = pr_curve(base_json, target_cat)
r_resc, p_resc = pr_curve(resc_json, target_cat)

plt.figure(figsize=(6,5))
plt.plot(r_base, p_base, label="Baseline", lw=2)
plt.plot(r_resc, p_resc, label="Rescored (α=0.50)", lw=2)
plt.xlabel("Recall"); plt.ylabel("Precision")
plt.title(f"PR Curve (cat_id={target_cat})")
plt.legend(); plt.grid(True, alpha=0.3); plt.show()

In [ ]:
# ===== Final Report Cell: summary.md + nice Markdown table + CSV =====
import json, os, pandas as pd
from pathlib import Path
from IPython.display import Markdown, display

RES_DIR = Path("results")
base_path = RES_DIR / "baseline_metrics.json"
abl_path  = RES_DIR / "shape_prior_ablation.json"

assert base_path.exists(), "Missing results/baseline_metrics.json. Run baseline eval cell first."
assert abl_path.exists(),  "Missing results/shape_prior_ablation.json. Run ablation cell first."

with open(base_path) as f:
    base = json.load(f)
with open(abl_path) as f:
    abla = json.load(f)

# Build a results DataFrame (segm only is the key metric for MS R-CNN, but we include bbox too in baseline.json)
rows = []
rows.append({
    "Method": "Baseline (MS R-CNN)",
    "AP@[.50:.95]": base["segm"]["AP@[.50:.95]"],
    "AP@.50":       base["segm"]["AP@.50"],
    "AP@.75":       base["segm"]["AP@.75"],
    "AP_small":     base["segm"]["AP_small"],
    "AP_medium":    base["segm"]["AP_medium"],
    "AP_large":     base["segm"]["AP_large"],
})

for k, m in abla.items():
    rows.append({
        "Method": f"Rescore {k}",
        "AP@[.50:.95]": m["AP@[.50:.95]"],
        "AP@.50":       m["AP@.50"],
        "AP@.75":       m["AP@.75"],
        "AP_small":     m["AP_small"],
        "AP_medium":    m["AP_medium"],
        "AP_large":     m["AP_large"],
    })

df = pd.DataFrame(rows)
# Compute deltas vs baseline
base_row = df.iloc[0]
for col in ["AP@[.50:.95]","AP@.50","AP@.75","AP_small","AP_medium","AP_large"]:
    df[f"Δ {col}"] = df[col] - float(base_row[col])

# Pretty print
pd.options.display.float_format = "{:,.4f}".format
df_sorted = df.sort_values(by="AP@[.50:.95]", ascending=False).reset_index(drop=True)

# Save CSV
csv_path = RES_DIR / "summary.csv"
df_sorted.to_csv(csv_path, index=False)

# Create Markdown table
def to_md_table(pdf):
    headers = list(pdf.columns)
    md = "| " + " | ".join(headers) + " |\n"
    md += "| " + " | ".join(["---"]*len(headers)) + " |\n"
    for _, r in pdf.iterrows():
        md += "| " + " | ".join(f"{r[c]:.4f}" if isinstance(r[c], (float,int)) else str(r[c]) for c in headers) + " |\n"
    return md

# Compose summary.md
best = df_sorted.iloc[0]
summary_lines = []

summary_lines.append("# MS R‑CNN: Baseline vs. Score Re‑weighting Ablation (segm AP)\n")
summary_lines.append(f"**Model**: MS R‑CNN R50‑FPN (1×) — **Dataset**: COCO 2017 val\n")
summary_lines.append(f"**Baseline segm AP@[.50:.95]**: {base_row['AP@[.50:.95]']:.4f}\n")
summary_lines.append(f"**Best variant**: {best['Method']}  →  **AP@[.50:.95]** = {best['AP@[.50:.95]']:.4f}  "
                     f"(Δ = {best['Δ AP@[.50:.95]']:+.4f})\n")
summary_lines.append("### Detailed Table\n")
summary_lines.append(to_md_table(df_sorted))

# Add quick notes section scaffold (you can edit in-place later)
summary_lines.append("\n### Notes\n- The improvement re‑weights detection scores using a shape prior (mask_area / bbox_area) with α‑sweep.\n"
                     "- Gains typically show at higher IoUs (AP@.75) if masks with better geometry get ranked higher.\n"
                     "- All runs used the same backbone/schedule; only scores were re‑weighted during evaluation.\n")

summary_md = "\n".join(summary_lines)
md_path = RES_DIR / "summary.md"
with open(md_path, "w") as f:
    f.write(summary_md)

display(Markdown(summary_md))
print(f"\nSaved summary files:\n- {md_path}\n- {csv_path}")

In [ ]:
# ===== Make a timestamped zip AND a fixed-named copy =====
import os, json, zipfile, time, shutil
from pathlib import Path
from IPython.display import FileLink, display

ROOT = Path(".")
RES_DIR = ROOT / "results"
RES_DIR.mkdir(exist_ok=True, parents=True)

# Assemble files same as before
candidates = [
    RES_DIR / "baseline_metrics.json",
    RES_DIR / "run_meta.json",
    RES_DIR / "shape_prior_ablation.json",
    RES_DIR / "msrcnn_r50_fpn_1x.bbox.json",
    RES_DIR / "msrcnn_r50_fpn_1x.segm.json",
]
candidates += sorted(RES_DIR.glob("msrcnn_r50_fpn_1x.segm.alpha*.json"))
candidates += sorted(RES_DIR.glob("*.png"))
candidates += sorted(RES_DIR.glob("*.jpg"))

files = [p for p in candidates if p.exists() and p.is_file()]
if not files:
    print("No result files found in ./results. Nothing to zip.")
else:
    ts = time.strftime("%Y%m%d-%H%M%S")
    timestamped = RES_DIR / f"msrcnn_outputs_{ts}.zip"
    with zipfile.ZipFile(timestamped, "w", compression=zipfile.ZIP_DEFLATED) as zf:
        for f in files:
            zf.write(f, f.relative_to(ROOT))

    fixed_zip = RES_DIR / "msrcnn_outputs_latest.zip"
    # copy (safer than symlink for downloads)
    shutil.copy2(timestamped, fixed_zip)

    print(f"Created: {timestamped.name} and copied to {fixed_zip.name}")
    display(FileLink(str(fixed_zip)))
    # If you also want the timestamped one linked:
    display(FileLink(str(timestamped)))

🔎 What’s still needed to finish Option 2 (and how to do it)
1) Baselines & sanity checks

Report MS R‑CNN baseline segm metrics and confirm they’re reasonable vs. model‑zoo (R50‑FPN 1× roughly ~36.0 mask AP; box AP ~38.2, depending on config/runtime). This proves your setup matches the standard implementation. [github.com]
(Optional but nice) Compare to plain Mask R‑CNN (same backbone/schedule) to show the paper‑level gain first, then your gain on top. (MMDetection has Mask R‑CNN configs next to MS R‑CNN.) [github.com]

2) Define the improvement clearly

State the hypothesis: ranking masks using a shape prior should better align scores with plausible mask geometry → improved segm AP (especially AP@0.75 or small/medium). Tie this back to the paper’s motivation (misalignment between class confidence and mask quality). [github.com]
Document the formula you implemented:
snew=sorigα⋅(mask_areabbox_area)1−α,α∈{0.25,0.5,0.75}s_{\text{new}} = s_{\text{orig}}^\alpha \cdot \left(\frac{\text{mask\_area}}{\text{bbox\_area}}\right)^{1-\alpha},\quad \alpha\in\{0.25,0.5,0.75\}snew​=sorigα​⋅(bbox_areamask_area​)1−α,α∈{0.25,0.5,0.75}


3) Ablation study

You already sweep α\alphaα. Present a small table with Baseline vs. α∈{0.25, 0.5, 0.75} on segm AP@[.50:.95], AP50, AP75, APs/m/l.
Interpret where you gain (e.g., AP75 ↑ suggests better ranking of higher‑quality masks).
Save the table to results/summary.csv or include in the notebook.

4) Error analysis (1–2 plots)
Include one or two plots to explain why the improvement helps:

PR curve on a frequent class (e.g., person) — Baseline vs. best α variant.
Score vs shape ratio scatter (you already have the cell) to visually show the heuristic’s signal.

5) Reproducibility

Pin the exact config & checkpoint used (your code already saves them in results/run_meta.json).
Note hardware/runtime (Kaggle P100/T4/A100; FPS if you measured).
Include a single “Run All” section at the top: install → download COCO (if needed) → evaluate baseline → ablation → visualize → zip outputs.

6) Short write‑up (4–6 pages or a compact README)
In your report (or repo README.md), include:

Problem & paper: Briefly summarize MS R‑CNN and why mask score calibration matters; cite the paper. [github.com]
Codebase: State you used MMDetection’s MS R‑CNN (link & cite), config, and checkpoint. [github.com]
Method: Your scoring formula, the rationale, and any assumptions/limits.
Experiments: Dataset split (COCO 2017 val), metrics (bbox & segm), baseline numbers vs. model‑zoo, and your ablation table.
Results: Best α improvement (absolute & relative), qualitative examples, PR curve.
Compute & reproducibility: environment, how to run, where outputs live.
Limitations & next steps: See ideas below.

7) Deliverables checklist (so you can submit confidently)

 Notebook/Script that runs end‑to‑end.
 Baseline bbox & segm metrics saved to results/baseline_metrics.json.
 Ablation metrics saved to results/shape_prior_ablation.json and a summary table.
 Visuals (a few PNGs/JPGs) in results/.
 Downloadable ZIP (msrcnn_outputs_latest.zip) with all artifacts.
 Short PDF report (or well‑structured README) with citations and your findings.

If you tick all of the above, you’ve met Option 2 requirements: you used a 2019 paper and implemented an improvement on top of its existing code base (MMDetection MS R‑CNN)